In [1]:
!pip install transformers datasets torch


In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DistilBertConfig

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")

config = DistilBertConfig.from_pretrained(
    "distilbert-base-uncased", 
    attention_probs_dropout_prob=0.4,  # Higher dropout for attention
    hidden_dropout_prob=0.4           # Higher dropout for hidden layers
)

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english", config=config)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [5]:
from datasets import load_dataset


ds = load_dataset("jahjinx/IMDb_movie_reviews")

README.md:   0%|          | 0.00/4.24k [00:00<?, ?B/s]

IMDB_train.csv:   0%|          | 0.00/47.6M [00:00<?, ?B/s]

IMDB_validation.csv:   0%|          | 0.00/5.23M [00:00<?, ?B/s]

IMDB_test.csv:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/36000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [6]:
ds["train"].to_csv("IMDB_train.csv")
ds["validation"].to_csv("IMDB_validation.csv")
ds["test"].to_csv("IMDB_test.csv")


Creating CSV from Arrow format:   0%|          | 0/36 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

13035905

In [7]:
import pandas as pd

train_df = pd.DataFrame(ds["train"])
validation_df = pd.DataFrame(ds["validation"])
test_df = pd.DataFrame(ds["test"])

# Combine datasets
complete_df = pd.concat([train_df, validation_df, test_df], ignore_index=True)
complete_df.to_csv("complete_IMDB_dataset.csv", index=False)


In [8]:
complete_df

,text,label
0,"Beautifully photographed and ably acted, gener...",0
1,"Well, where to start describing this celluloid...",0
2,I first caught the movie on its first run on H...,1
3,I love Umberto Lenzi's cop movies -- ROME ARME...,0
4,I generally won't review movies I haven't seen...,0
...,...,...
49995,I liked this movie. That's pretty much all I c...,1
49996,"Imagine the plight of Richard, a painter, whos...",1
49997,There is an inherent problem with commenting o...,0
49998,When The Matrix appeared in 1999 and questione...,1


In [9]:
from sklearn.model_selection import train_test_split

# Split into training, validation, and test subsets
train_df, temp_df = train_test_split(complete_df, train_size=0.7, random_state=42)
val_df, test_df = train_test_split(temp_df, train_size=0.5, random_state=42)

print(f"Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")

Train: 35000, Validation: 7500, Test: 7500


In [10]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)


In [11]:
def tokenize_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

encoded_train = train_dataset.map(tokenize_function, batched=True)
encoded_val = val_dataset.map(tokenize_function, batched=True)
encoded_test = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2  # Positive and Negative sentiment labels
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,  # Train for only 1 epoch
    weight_decay=0.1,
    logging_dir="./logs",
    logging_steps=200,
    save_total_limit=2,
    save_strategy="epoch",
    fp16=True,
    report_to="none",
    load_best_model_at_end=True,  # Still useful if you save checkpoints
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    tokenizer=tokenizer,


)


<ipython-input-16-76816ea5a368>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [27]:
!nvidia-smi


Wed Jan 29 12:14:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P0             30W /  250W |    1797MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [18]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.386500,0.374528
2,0.362700,0.362805


TrainOutput(global_step=4376, training_loss=0.41094621748985277, metrics={'train_runtime': 490.2953, 'train_samples_per_second': 142.771, 'train_steps_per_second': 8.925, 'total_flos': 2318179476480000.0, 'train_loss': 0.41094621748985277, 'epoch': 2.0})

In [28]:
trainer.evaluate()


{'eval_loss': 0.36280521750450134,
 'eval_runtime': 14.5969,
 'eval_samples_per_second': 513.808,
 'eval_steps_per_second': 32.13,
 'epoch': 2.0}

In [25]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Save model and tokenizer using the Hugging Face API
model.save_pretrained("./IMDB_fine_tuned_model")
tokenizer.save_pretrained("./IMDB_fine_tuned_model_tokenizer")


('./IMDB_fine_tuned_model_tokenizer/tokenizer_config.json',
 './IMDB_fine_tuned_model_tokenizer/special_tokens_map.json',
 './IMDB_fine_tuned_model_tokenizer/vocab.txt',
 './IMDB_fine_tuned_model_tokenizer/added_tokens.json',
 './IMDB_fine_tuned_model_tokenizer/tokenizer.json')

In [26]:
!zip -r IMDB_fine_tuned_model.zip ./IMDB_fine_tuned_model
!zip -r IMDB_fine_tuned_model_tokenizer.zip ./IMDB_fine_tuned_model_tokenizer


  adding: IMDB_fine_tuned_model/ (stored 0%)
  adding: IMDB_fine_tuned_model/vocab.txt (deflated 53%)
  adding: IMDB_fine_tuned_model/special_tokens_map.json (deflated 42%)
  adding: IMDB_fine_tuned_model/tokenizer.json (deflated 71%)
  adding: IMDB_fine_tuned_model/model.safetensors (deflated 8%)
  adding: IMDB_fine_tuned_model/training_args.bin (deflated 51%)
  adding: IMDB_fine_tuned_model/tokenizer_config.json (deflated 75%)
  adding: IMDB_fine_tuned_model/config.json (deflated 46%)
  adding: IMDB_fine_tuned_model_tokenizer/ (stored 0%)
  adding: IMDB_fine_tuned_model_tokenizer/vocab.txt (deflated 53%)
  adding: IMDB_fine_tuned_model_tokenizer/special_tokens_map.json (deflated 42%)
  adding: IMDB_fine_tuned_model_tokenizer/tokenizer.json (deflated 71%)
  adding: IMDB_fine_tuned_model_tokenizer/tokenizer_config.json (deflated 75%)


In [29]:


from transformers import AutoModelForSequenceClassification

# Save the fine-tuned model explicitly
model.save_pretrained("./IMDB_fine_tuned_model")


In [30]:
!zip -r IMDB_fine_tuned_model.zip ./IMDB_fine_tuned_model


updating: IMDB_fine_tuned_model/ (stored 0%)
updating: IMDB_fine_tuned_model/vocab.txt (deflated 53%)
updating: IMDB_fine_tuned_model/special_tokens_map.json (deflated 42%)
updating: IMDB_fine_tuned_model/tokenizer.json (deflated 71%)
updating: IMDB_fine_tuned_model/model.safetensors (deflated 8%)
updating: IMDB_fine_tuned_model/training_args.bin (deflated 51%)
updating: IMDB_fine_tuned_model/tokenizer_config.json (deflated 75%)
updating: IMDB_fine_tuned_model/config.json (deflated 46%)
